In [81]:
import pandas as pd
import sqlite3
from selenium import webdriver
import math
from email.mime.multipart import MIMEMultipart
import sys
from email.MIMEText import MIMEText

## <font color=Red>Instruction</font>


- <font color=blue> Create the Sqlite database by executing the followinf script only once</font>
-  <font color=blue>Verify the email settings in sendmail functions ( incase you need to change the email)</font>
- <font color=blue> Package the code in a py script and add it to the scheduler</font>


In [3]:
# this ned to be exeute only once


#conn = sqlite3.connect("data.db")
#cur = conn.cursor()
#cur.execute("""CREATE TABLE status_table 
#        (College TEXT, Status TEXT, Date TEXT )""")
#cur.close()
#conn.close()

In [69]:
#utility functions
def convert_date_sqilte(date):
    d,m,y = date.split('/',3)
    return y+'-'+m+'-'+d

reporting_cycle = ['Spring','Fall/Annual','Summer/Summer I']
CBMS = ['CBM001','CBM004','CBM008','CBM00S','CBM009','CBM0E1','CBM00B','FAD001']
years = ['2015','2016','2017']

def sendemail(Status):
    if Status:
        part1 = '<h2 style="text-align: center;">Summary of THECB Bot</h2><p>New changes to THECB certification dates:</p>'
    else:
        part1 = '<h2 style="text-align: center;">Summary of THECB Bot</h2><p>There are no changes to certification dates.</p>'
        
    html = new_df.to_html()
    html = part1 +html
    body = MIMEText(html, 'html')

    me="THCB WEBSCRAPER <amosharraf@utsystem.edu>"
    recipients  = ["arashaga@gmail.com","amosharraf@utsystem.edu"]

    try:

        msg = MIMEMultipart('alternative')
        msg.attach(body)
        msg['Subject']= 'Hi'
        msg['From']   = me # some SMTP servers will do this automatically, not all
        msg['to'] = ", ".join(recipients)

        conn = smtplib.SMTP('outlook.utsystem.edu',25)
        conn.set_debuglevel(0)

        conn.sendmail(me,recipients, msg.as_string())
        conn.quit()

    except:
        sys.exit( "mail failed; %s" % "CUSTOM_ERROR" ) 
        conn.quit() # give an error message

In [114]:
driver = webdriver.Chrome(executable_path="c:/chromedriver.exe")
new_df = None
issues = []
c = []
d = []
s = []
n = []
conn = sqlite3.connect("data.db")
cur = conn.cursor()
for rc in reporting_cycle:
    for year in years:
        for cbm in CBMS:
            driver.get("http://www.txhighereddata.org/Interactive/CBMStatus")
            query = "//select[@name='RepCyc']/option[text()='"+ rc+ "']"
            element=driver.find_element_by_xpath(query).click()
            
            query = "//select[@name='RepYr']/option[text()='"+year+"']"
            element=driver.find_element_by_xpath(query).click()
            
            element=driver.find_element_by_name("CBMRep")
            query = "//input[@value='"+cbm+"']"
            element = driver.find_element_by_xpath(query).click()

            driver.find_element_by_xpath("//input[@value='Get Status']").click()
            element = driver.find_element_by_class_name("divprime")
            element.get_attribute("id")
            page = element.get_attribute('innerHTML')
            table = pd.read_html(page)
            df =table[2][2:]
            if len(df.columns) != 3:
                st = 'reporting cycle: '+rc+'CBM: '+cbm,'year: '+year
                issues.append(st)
                break
            df.columns = ['College','Status','Date']
            df = df[df['College'].str.contains('The University of Texas')]
            df = df.reset_index(drop=True)



            for index, row in df.iterrows():
                st = row['College']
                #for sqlite you need to have the mm-dd-yyy format
                #print('MISSING DATE ','reporting cycle: ',rc,'CBM: ',cbm,'year: ',year)
                if type(row['Date']) == float:
                    break
                date = convert_date_sqilte(row['Date']) 
                #TEST CASE
                #if index == 3:
                #    date = '05/07/2008'

                # if the new date is not found in the database
                sql = 'select * from status_table where College= ? and Date= ?'

                temp = pd.read_sql_query(sql,conn,params=(st,date,))
                tqsql = 'Select * from status_table s1 where \
                                  Date=(select Max(Date) from status_table s2 where s1.College=s2.College) \
                                  and College = ?'
                tq = pd.read_sql_query(tqsql,conn,params=(st,))
                
                # if the record is not there insert it and prepare the new_df to be send out as a notification
                if temp.empty:
                    values = [row['College'],row['Status'],convert_date_sqilte(row['Date'])]
                    cur.execute("insert into status_table values (?, ?, ?)", values)
                    print('Data: ',row['College'],row['Date'],row['Status'])
                    c.append(row['College'])
                    s.append(row['Status'])
                    d.append(row['Date'])
                    if not tq.empty:
                        n.append(tq['Date'].values[0])
                    else:
                        n.append(None)
                    
dic = {'College':c,'Status':s,'New Date':d,'Old Date': n}    

new_df = pd.DataFrame.from_dict(dic)
conn.commit()
cur.close()
conn.close()
driver.close()
print(issues)
if new_df.empty:
    sendemail(False)
else:
    sendemail(True)

Data:  The University of Texas at Dallas 01/30/2018 CERTIFIED
Data:  The University of Texas at Arlington 03/27/2018 CERTIFIED
Data:  The University of Texas at Austin 03/27/2018 CERTIFIED
Data:  The University of Texas at Dallas 03/29/2018 CERTIFIED
Data:  The University of Texas at El Paso 03/29/2018 CERTIFIED
Data:  The University of Texas-Rio Grande Valley 03/27/2018 CERTIFIED
[('reporting cycle: SpringCBM: CBM009', 'year: 2015'), ('reporting cycle: SpringCBM: CBM009', 'year: 2016'), ('reporting cycle: SpringCBM: CBM009', 'year: 2017'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2015'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2016'), ('reporting cycle: Summer/Summer ICBM: CBM009', 'year: 2017')]
